2023.05.11 数据治理
mRNA 分类模型构建
参考Nature cfRNA文章和代码 
https://github.com/miramou/pe_cfrna/tree/master/analysis

# 数据加载

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz 
from sklearn import metrics
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
import eli5 
from eli5.sklearn import PermutationImportance

In [2]:
import shap 
from pdpbox import pdp, info_plots 
np.random.seed(2022) 

pd.options.mode.chained_assignment = None  
%matplotlib inline

# Functions

In [27]:
#查看当下路径
import os
print(os.getcwd())
os.chdir("/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data")
print(os.getcwd())

/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data
/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data


In [8]:
from util.gen_utils import *

In [10]:
from util.qc_util import *
from util.qpcr_utils import *

In [11]:
import os

%matplotlib inline

%load_ext autoreload
%autoreload 2

# Data clean

## discovery组数据治理

In [26]:
# discovery组数据治理
# 导入数据discovery_C = dc
dc = pd.read_csv('GSE36961_exprSet_filtered_final.csv', index_col=0)
# 输出首尾五行
dc.head()

,X7A5,A1BG,A1CF,A26A1,A26B1,A26C1B,A26C3,A2BP1,A2M,A2ML1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
653513_mRNA,65.29057,81.10231,81.45553,64.55132,57.29808,77.69164,80.02750,122.6194,1171.7710,64.14913,...,86.12190,69.57877,81.26176,85.64597,97.20049,65.23239,1321.857,300.3123,206.2749,334.9858
647527_mRNA,62.31787,69.69264,67.15081,71.03593,70.17850,79.60251,72.63081,106.0079,681.6916,73.47318,...,73.89655,61.87214,72.54843,98.17334,95.65656,66.31665,1471.323,302.0973,243.8547,347.4308
634021_mRNA,61.56281,70.10897,70.21436,61.28361,65.27797,65.27431,70.85375,136.9092,551.3333,61.11268,...,72.13641,60.92840,68.41888,90.36416,91.79478,67.51790,1122.348,266.0344,157.5872,224.1300
43_mRNA,60.84670,60.96588,75.43973,63.37952,50.81472,65.37705,70.37589,102.9477,1317.8960,61.97736,...,76.27374,64.15804,67.91789,92.85126,99.04301,59.35860,1350.857,322.1322,231.0520,261.3479
5059_mRNA,64.96912,70.82362,67.28829,66.16459,52.81135,75.51786,70.73528,119.3004,1124.1230,67.10345,...,101.84010,64.05208,64.70481,102.06990,98.46761,60.17641,1076.411,641.0287,217.3451,219.7906


In [1]:
#导入angiogenesis genelist
angiogenesis = pd.read_csv('angiogenesis_protein.csv')
# 输出首尾五行
angiogenesis.head()
rows_to_drop = ['CXCL8', 'TYMP', 'ECSCR', 'NDNF', 'PIK3R6', 'ACKR3', 'AIMP1', 'C19orf10', 'NAA15']
angiogenesis = angiogenesis[~angiogenesis['X.node'].isin(rows_to_drop)]

NameError: name 'pd' is not defined

In [29]:
#angiogenesis中X.node列 提取 dc 中 的列数据
angiogenesis.set_index('X.node', inplace=True)
angiogenesis_genes = angiogenesis.index.values
dc_ani = dc.loc[:, angiogenesis_genes]
dc_ani.head()
# 5 rows × 175 columns

,VEGFA,FN1,EGF,KDR,PECAM1,FGF2,ITGB1,NOTCH1,HIF1A,TEK,...,MEOX2,OVOL2,VEZF1,CALCRL,EGR3,ELK3,RTN4,SLC12A6,THSD7A,ZNF304
653513_mRNA,215.3208,72.10228,79.49622,81.47382,397.2655,87.55611,1967.808,345.0295,165.5901,449.1410,...,98.21214,67.89413,1540.014,150.5532,85.54486,71.91047,508.3564,74.95110,66.65141,74.27869
647527_mRNA,233.5941,60.83722,79.64598,81.01164,280.4375,76.69513,1760.400,292.6602,129.4617,343.2925,...,85.22502,78.97446,1647.732,118.6783,97.84703,71.56583,628.2866,71.56470,54.12376,78.00294
634021_mRNA,180.6582,63.64766,75.90014,74.75217,269.5177,68.91901,1227.290,313.0690,102.2128,380.1647,...,80.58092,78.61082,1447.595,98.6368,84.85375,64.81683,193.3965,63.59631,48.99234,68.69955
43_mRNA,198.6208,64.54980,71.41699,83.72794,287.8640,83.03422,1580.264,322.4279,120.7920,374.5043,...,76.08558,62.40175,1433.261,120.7141,77.63084,73.01337,424.5472,66.36166,61.68090,66.16611
5059_mRNA,199.9315,69.61796,72.64426,78.08111,677.6162,77.11231,1715.637,371.7405,172.9951,660.7704,...,95.73409,70.67875,1502.072,191.1847,72.23016,74.76708,960.0621,65.26451,54.46039,70.31483


In [32]:
#导入样本的组别信息 并把health 改成0，HCM 改成1
# 导入数据discovery_sample = ds
ds = pd.read_csv('GSE36961_groupInfo_final.csv', index_col=0)
# 输出首尾五行
ds.head()
# 重命名重命名 'sample type:ch1' 列为 'group'
ds = ds.rename(columns={'sample type:ch1': 'group'})
# 将 'control' 改为 0，'case' 改为 1
ds['group'] = ds['group'].replace({'control': 0, 'case': 1})
ds.head()

,description,age (yrs):ch1,disease state:ch1,group,Sex:ch1
7_mRNA,7_mRNA,9.0,hypertrophic cardiomyopathy (HCM),1,male
R26_mRNA,R26_mRNA,10.0,hypertrophic cardiomyopathy (HCM),1,male
670934_mRNA,670934_mRNA,10.0,hypertrophic cardiomyopathy (HCM),1,female
NM10_mRNA,NM10_mRNA,11.0,hypertrophic cardiomyopathy (HCM),1,male
666295_mRNA,666295_mRNA,13.0,hypertrophic cardiomyopathy (HCM),1,female


In [72]:
merged = pd.merge(dc_ani,ds, left_index=True, right_index=True)
merged.drop(['description', 'age (yrs):ch1', 'disease state:ch1', 'Sex:ch1'], axis=1, inplace=True)
merged.head()

#导出merge的数据到 /share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/discovery中
merged.to_csv('./discovery/discovery.csv', index=True)

## validation1组数据治理

In [42]:
# validation1组数据治理
# 导入数据validation1_C = vc1
vc1 = pd.read_csv('GSE141910_filtered_matrix_final_55.csv', index_col=0)
# 输出首尾五行
vc1.head()

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC02928,CNNM3.DT,DGCR11,MICE,PAXIP1.DT,LINC02019,SFTPD.AS1,SLC16A4.AS1,TMEM167B.DT,APP.DT
C00055,11.563466,6.003549,13.433820,11.858282,9.368730,10.068701,14.105011,13.700063,13.143990,12.016234,...,5.727914,6.837539,8.398850,7.855026,10.083613,7.746773,9.280887,10.739730,7.955720,8.248171
C00192,11.815389,8.001471,13.467168,11.466473,9.777799,12.376141,14.461949,14.187085,12.987010,10.808826,...,5.254237,7.732284,7.656336,5.616807,10.631758,6.839200,7.938735,10.817570,8.713669,8.001471
C01631,11.559774,9.251097,13.362605,11.696218,9.712775,9.600785,15.447231,13.368317,13.187364,12.256240,...,4.121814,6.192203,7.865975,6.323447,10.196490,8.429242,8.788570,11.420105,8.959757,8.275619
C01862,12.000571,11.411732,12.811976,11.714120,10.073539,10.648653,17.024381,13.267972,12.930504,12.327137,...,2.873866,7.171547,7.858759,7.628754,10.352299,7.828063,8.619821,11.444987,8.655226,6.146885
C02242,11.609025,7.976349,13.571295,11.722775,9.578087,9.674840,14.081200,13.310249,13.505950,12.284288,...,4.465387,6.787315,7.635312,6.898346,10.288117,7.187853,7.924818,10.947590,8.410245,8.120739


In [54]:
#导入angiogenesis genelist
angiogenesis = pd.read_csv('angiogenesis_protein.csv')
# 输出首尾五行
angiogenesis.head()
angiogenesis.tail()
# KeyError: "['PECAM1', 'CTGF', 'FIGF', 'SHH', 'FGF6', 'GDF2', 'ANGPTL3', 'HRG', 'NOV', 'EPGN', 'WARS', 'CXCL17', 'GPR56', 
# 'TMEM2', 'CD160', 'PIK3R6', 'OVOL2', 'C19orf10'] not in index"
rows_to_drop = ['PECAM1', 'CTGF', 'FIGF', 'SHH', 'FGF6', 'GDF2', 'ANGPTL3', 'HRG', 'NOV', 'EPGN', 'WARS', 'CXCL17', 'GPR56',
               'TMEM2', 'CD160', 'PIK3R6', 'OVOL2', 'C19orf10']
angiogenesis = angiogenesis[~angiogenesis['X.node'].isin(rows_to_drop)]
angiogenesis.tail()

,X.node
179,NAA15
180,RTN4
181,SLC12A6
182,THSD7A
183,ZNF304


In [55]:
#angiogenesis中X.node列 提取 dc 中 的列数据
angiogenesis.set_index('X.node', inplace=True)
angiogenesis_genes = angiogenesis.index.values

In [56]:
vc1_ani = vc1.loc[:, angiogenesis_genes]
vc1_ani.head()
# 5 rows × 175 columns

,VEGFA,FN1,EGF,KDR,FGF2,ITGB1,NOTCH1,HIF1A,TEK,PDGFRB,...,VEZF1,AIMP1,CALCRL,EGR3,ELK3,NAA15,RTN4,SLC12A6,THSD7A,ZNF304
C00055,15.641355,16.656666,12.367969,13.372909,11.812023,17.147791,11.638597,14.484383,14.009661,14.197021,...,14.299567,13.624326,14.020841,8.792870,13.619218,13.721076,14.940059,12.554797,9.841931,12.225661
C00192,14.980942,16.203126,11.624302,12.176503,11.573587,15.788818,11.020529,14.112662,12.459507,13.603597,...,13.940237,13.663022,13.403258,10.942737,13.230699,13.040273,15.890991,9.847762,9.161128,10.934722
C01631,15.725347,17.107051,11.371638,13.964478,11.634356,16.556285,11.818086,14.154961,13.793178,14.695271,...,14.164841,13.577552,14.010861,10.004457,14.012686,13.426963,16.089832,11.634356,11.146399,12.077173
C01862,14.416737,18.468589,11.348248,13.238366,12.347572,16.952879,11.145329,14.467102,13.131647,15.293914,...,14.114459,13.270114,14.008934,8.527309,13.673957,13.058329,16.237591,12.300830,11.326451,11.865388
C02242,15.806238,15.780423,11.571819,14.658419,12.416088,16.886136,11.745229,14.221943,13.344970,14.385144,...,13.986202,13.778723,13.572342,11.417254,13.678248,13.699486,15.506087,12.039791,9.891651,12.385442


In [64]:
#导入样本的组别信息 并把health 改成0，HCM 改成1
# 导入数据validation1_sample = vs1
vs1 = pd.read_csv('GSE141910_sample_HCM_55.csv', index_col=0)
# 输出首尾五行
vs1.head()
# 重命名重命名 'sample type:ch1' 列为 'group'
vs1 = vs1.rename(columns={'etiology:ch1': 'group'})
# 将 'control' 改为 0，'case' 改为 1
vs1['group'] = vs1['group'].replace({'Non-Failing Donor': 0, 'Hypertrophic cardiomyopathy (HCM)': 1})

# 把行名改成title
vs1.set_index('title', inplace=True)
vs1.head()

,age:ch1,group,race:ch1,Sex:ch1
title,,,,
C01862,21,1,Caucasian,Female
C02351,30,1,Caucasian,Female
C01631,38,1,Caucasian,Female
P01027,46,1,African American,Female
C02242,48,1,Caucasian,Female


In [71]:
merged = pd.merge(vc1_ani,vs1, left_index=True, right_index=True)
merged.drop([ 'age:ch1', 'race:ch1', 'Sex:ch1'], axis=1, inplace=True)
merged.head()

#导出merge的数据到 /share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/discovery中
merged.to_csv('./validation1/validation1.csv', index=True)
print(merged.tail())
print(merged.shape) 

            VEGFA        FN1        EGF        KDR       FGF2      ITGB1  \
P01580  15.626101  17.334014  11.943400  13.689446  12.291533  17.431199   
P01595  15.387644  15.987901  11.469417  14.835424  12.754659  17.109145   
P01602  15.719810  16.129752  11.473629  13.481308  11.524827  16.690873   
P01630  13.622448  15.786812   9.892322  11.292663  10.613606  13.377517   
P01631  13.762831  15.942996  10.097311  11.726441  10.689729  14.014134   

           NOTCH1      HIF1A        TEK     PDGFRB  ...      AIMP1     CALCRL  \
P01580  11.138247  14.837897  14.207780  14.692508  ...  13.628616  13.497343   
P01595  11.138614  14.633315  14.010343  14.047772  ...  13.739633  14.320706   
P01602  11.890443  14.217331  13.815321  14.621273  ...  13.573718  13.581433   
P01630  12.718126  12.100721  11.716291  13.765904  ...  13.264069  11.103530   
P01631  12.210486  13.156802  12.463406  14.033796  ...  13.379067  11.815467   

             EGR3       ELK3      NAA15       RTN4    SL

# validation2组数据治理

In [73]:
# validation2组数据治理
# 导入数据validation2_C = vc2
vc2 = pd.read_csv('GSE160997_filtered_matrix_final.csv', index_col=0)
# 输出首尾五行
print(vc2.head())

            TNMD  DPM1  SCYL3  C1orf112  FGR   CFH  FUCA2  GCLC  NFYA  STPG1  \
GSM4887610     6   152    101        19   33   807    287   327   136     37   
GSM4887611     2   306    107        22   41   515    249   290   123     32   
GSM4887612     1   261    100        29   24   719    253   239   139     46   
GSM4887613     1   209     84        14   13   558    157   251    89     31   
GSM4887614     2   251    103        19   74  1185    344   335   135     38   

            ...  RNU7.1  SNORD2  SNORA47  SCARNA10  SNORD108  SNORD13  \
GSM4887610  ...       4       3        5       393       104       11   
GSM4887611  ...       1      11       11       838       148        9   
GSM4887612  ...       3       6        5       352       120       29   
GSM4887613  ...       6       4       11       192        74        6   
GSM4887614  ...       6       5       16       602        56       37   

            RNU7.73P  RNU7.119P  SNORA59A  SNORA84  
GSM4887610         1       

In [94]:
#导入angiogenesis genelist
angiogenesis = pd.read_csv('angiogenesis_protein.csv')
# 输出首尾五行
angiogenesis.head()
print(angiogenesis.tail())

rows_to_drop = ['PECAM1', 'CTGF', 'FIGF', 'SHH', 'LEP', 'FGF6', 'EREG', 'GDF2', 'TDGF1', 'HRG', 
'NOV', 'ECSCR', 'EPGN', 'TNFSF12', 'PDCD6', 'WARS', 'GPR56', 'TMEM2', 'PIK3R6', 'OVOL2', 'C19orf10']
angiogenesis = angiogenesis[~angiogenesis['X.node'].isin(rows_to_drop)]
print(angiogenesis.tail())

      X.node
179    NAA15
180     RTN4
181  SLC12A6
182   THSD7A
183   ZNF304
      X.node
179    NAA15
180     RTN4
181  SLC12A6
182   THSD7A
183   ZNF304


In [95]:
#angiogenesis中X.node列 提取 dc 中 的列数据
angiogenesis.set_index('X.node', inplace=True)
angiogenesis_genes = angiogenesis.index.values

In [96]:
vc2_ani = vc2.loc[:, angiogenesis_genes]
vc2_ani.head()
# 5 rows × 175 columns

,VEGFA,FN1,EGF,KDR,FGF2,ITGB1,NOTCH1,HIF1A,TEK,PDGFRB,...,VEZF1,AIMP1,CALCRL,EGR3,ELK3,NAA15,RTN4,SLC12A6,THSD7A,ZNF304
GSM4887610,814,4427,348,941,110,8307,331,1034,524,1677,...,1031,261,665,9,177,783,1629,560,40,180
GSM4887611,849,8621,272,1379,131,10517,365,1064,857,1201,...,767,284,249,6,150,822,2245,539,64,218
GSM4887612,969,113718,131,752,100,27285,370,1580,680,1804,...,733,252,506,30,236,1089,2968,586,49,216
GSM4887613,644,3651,322,992,74,7373,186,673,451,971,...,584,218,531,1,150,623,242,391,34,151
GSM4887614,1063,43866,208,877,192,11424,494,3397,709,2072,...,890,278,474,21,244,716,1380,647,79,224


In [99]:
#导入样本的组别信息 并把health 改成0，HCM 改成1
# 导入数据validation1_sample = vs2
vs2 = pd.read_csv('GSE160997_group_final.csv', index_col=0)
# 输出首尾五行
print(vs2.head())
# 重命名重命名 'sample type:ch1' 列为 'group'
vs2 = vs2.rename(columns={'etiology:ch1': 'group'})
# 将 'control' 改为 0，'case' 改为 1
vs2['group'] = vs2['group'].replace({'Control': 0, 'HCM': 1})
print(vs2.head())

           sampleID    group  groupID                            other
sample                                                                
GSM4887610    ctrl1  Control        0  anterior septal tissues - ctrl1
GSM4887611    ctrl2  Control        0  anterior septal tissues - ctrl2
GSM4887612    ctrl3  Control        0  anterior septal tissues - ctrl3
GSM4887613    ctrl4  Control        0  anterior septal tissues - ctrl4
GSM4887614    ctrl5  Control        0  anterior septal tissues - ctrl5
           sampleID  group  groupID                            other
sample                                                              
GSM4887610    ctrl1      0        0  anterior septal tissues - ctrl1
GSM4887611    ctrl2      0        0  anterior septal tissues - ctrl2
GSM4887612    ctrl3      0        0  anterior septal tissues - ctrl3
GSM4887613    ctrl4      0        0  anterior septal tissues - ctrl4
GSM4887614    ctrl5      0        0  anterior septal tissues - ctrl5


In [100]:
merged = pd.merge(vc2_ani,vs2, left_index=True, right_index=True)
merged.drop([ 'sampleID', 'groupID', 'other'], axis=1, inplace=True)
merged.head()

#导出merge的数据到 /share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/discovery中
merged.to_csv('./validation2/validation2.csv', index=True)
print(merged.tail())
print(merged.shape) 

            VEGFA    FN1  EGF  KDR  FGF2  ITGB1  NOTCH1  HIF1A  TEK  PDGFRB  \
GSM4887628   1273  16792  194  673   115   7682     400    884  385    1845   
GSM4887629   1096   1256   29  225    27    860     782    143  162     780   
GSM4887630   1239   7851  132  871   107   5528     481    580  368     987   
GSM4887631    979   5362  120  488   102   4698     382    628  274     807   
GSM4887632    767  13858  202  487   122   8947     473    922  425    1412   

            ...  AIMP1  CALCRL  EGR3  ELK3  NAA15  RTN4  SLC12A6  THSD7A  \
GSM4887628  ...    226     448    17   192    482   656      550      54   
GSM4887629  ...     40      70    17   126    140   132      162      30   
GSM4887630  ...    166     523    44   168    391   946      319      98   
GSM4887631  ...    187     252    43   118    442   490      285      30   
GSM4887632  ...    239     412    11   222    492  1589      746      71   

            ZNF304  group  
GSM4887628     188      1  
GSM4887629  

# validation3组数据治理

In [3]:
#查看当下路径
import os
print(os.getcwd())
os.chdir("/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data")
print(os.getcwd())

/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data
/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data


In [32]:
# validation3组数据治理
# 导入数据validation3_C = vc3
vc3 = pd.read_csv('GSE130036_filtered_matrix_final.csv', index_col=0)
# 输出首尾五行
vc3.tail()

,ARF5,M6PR,ESRRA,FKBP4,CYP26B1,NDUFAF7,FUCA2,DBNDD1,HS3ST1,SEMA3F,...,SCYGR8,SCYGR3,AL031430.1,TMDD1,AC117945.2,AC099499.2,AL031731.1,AL592182.2,AL031985.4,AL805961.1
ND1_LV,392.055,1542.900,451.327,936.191,17.1817,391.976,340.531,35.0109,112.7380,143.6950,...,0,0,41.4501,2,0,2.12014,4.12032,0,0,33.5458
ND2,654.060,1372.670,848.322,690.329,12.9950,442.966,330.111,58.1065,126.6190,90.9471,...,0,0,43.0551,9,0,1.05615,1.03121,0,0,26.6675
sc2_LV,430.305,1378.060,726.697,840.834,33.5455,399.524,324.167,25.7233,108.8920,114.3330,...,0,0,50.2264,4,0,4.82411,6.90292,0,2,42.1793
sc5_LV,697.970,927.463,651.411,462.016,35.6682,250.370,418.970,38.2672,46.1011,169.2860,...,0,0,47.5773,12,0,0.00000,4.15075,0,2,31.0978
sc6_LV,452.386,2039.070,1242.640,1823.150,26.6749,456.140,384.971,43.6647,89.2106,205.8610,...,0,0,37.3962,5,1,3.15197,9.43216,0,0,34.7352


In [13]:
#导入angiogenesis genelist
angiogenesis = pd.read_csv('angiogenesis_protein.csv')
# 输出首尾五行
angiogenesis.head()
print(angiogenesis.tail())
rows_to_drop = ['FGF2', 'NRP1', 'EFNB2', 'NRP2', 'EGFL7', 'ESM1', 'FZD8', 'C1GALT1','PECAM1', 'CTGF', 'FIGF', 'NOV', 'ECSCR', 'WARS', 'GPR56', 'TMEM2', 'C19orf10']
angiogenesis = angiogenesis[~angiogenesis['X.node'].isin(rows_to_drop)]
print(angiogenesis.tail())

      X.node
179    NAA15
180     RTN4
181  SLC12A6
182   THSD7A
183   ZNF304
      X.node
179    NAA15
180     RTN4
181  SLC12A6
182   THSD7A
183   ZNF304


In [14]:
#angiogenesis中X.node列 提取 dc 中 的列数据
angiogenesis.set_index('X.node', inplace=True)
angiogenesis_genes = angiogenesis.index.values

In [15]:
vc3_ani = vc3.loc[:, angiogenesis_genes]
vc3_ani.head()
# 5 rows × 175 columns

,VEGFA,FN1,EGF,KDR,ITGB1,NOTCH1,HIF1A,TEK,PDGFRB,FLT1,...,VEZF1,AIMP1,CALCRL,EGR3,ELK3,NAA15,RTN4,SLC12A6,THSD7A,ZNF304
HCM269,75.7254,0.0,461.096,1414.070,3584.84,3.04476,30.0692,334.102,735.901,955.86,...,1033.240,39.1630,529.860,69.0000,846.838,998.153,277.861,681.764,7.17191,319.692
HCM273,37.6605,0.0,381.455,978.909,4945.51,4.54240,17.4854,623.545,1153.430,1393.92,...,1036.140,20.3536,380.116,33.0000,1209.590,906.790,342.850,482.818,25.41640,262.259
HCM282,15.6717,0.0,412.791,924.741,7825.49,3.98671,102.7640,776.293,1079.290,1603.84,...,1322.520,41.6512,378.434,70.2374,1488.450,1502.000,668.593,821.119,9.47747,352.679
HCM395,62.8119,0.0,415.544,753.172,4832.22,5.74996,15.9566,665.929,978.295,1489.18,...,958.248,16.1132,419.580,35.5533,1097.870,792.813,310.143,505.360,4.90248,252.070
HCM405,43.4162,0.0,259.767,1996.450,6873.09,15.72030,40.1066,464.618,1078.700,1514.09,...,910.745,24.2780,628.907,56.0000,1264.100,819.821,386.455,570.512,0.00000,312.924


In [34]:
#导入样本的组别信息 并把health 改成0，HCM 改成1
# 导入数据validation1_sample = vs2
vs3 = pd.read_csv('GSE130036_groupInfo_final.csv', index_col=0)
# 输出首尾五行
print(vs3.head())
# 重命名重命名 'sample type:ch1' 列为 'group'
vs3 = vs3.rename(columns={'group:ch1': 'group'})
# 将 'control' 改为 0，'case' 改为 1
vs3['group'] = vs3['group'].replace({'Healthy': 0, 'hypertrophic cardiomyopathy': 1})
print(vs3)

#提取行名，groupID
extracted_data = vs3[['group']]


       geo_accession                 status submission_date last_update_date  \
HCM269    GSM3730500  Public on Apr 19 2019     Apr 18 2019      Apr 19 2019   
HCM273    GSM3730501  Public on Apr 19 2019     Apr 18 2019      Apr 19 2019   
HCM282    GSM3730502  Public on Apr 19 2019     Apr 18 2019      Apr 19 2019   
HCM395    GSM3730503  Public on Apr 19 2019     Apr 18 2019      Apr 19 2019   
HCM405    GSM3730504  Public on Apr 19 2019     Apr 18 2019      Apr 19 2019   

       type  channel_count    source_name_ch1  organism_ch1  \
HCM269  SRA              1  Myocardial tissue  Homo sapiens   
HCM273  SRA              1  Myocardial tissue  Homo sapiens   
HCM282  SRA              1  Myocardial tissue  Homo sapiens   
HCM395  SRA              1  Myocardial tissue  Homo sapiens   
HCM405  SRA              1  Myocardial tissue  Homo sapiens   

                       characteristics_ch1 characteristics_ch1.1  ...  \
HCM269  group: hypertrophic cardiomyopathy             Sex: male  .

In [35]:
merged = pd.merge(vc3_ani,extracted_data, left_index=True, right_index=True)
merged

,VEGFA,FN1,EGF,KDR,ITGB1,NOTCH1,HIF1A,TEK,PDGFRB,FLT1,...,AIMP1,CALCRL,EGR3,ELK3,NAA15,RTN4,SLC12A6,THSD7A,ZNF304,group
HCM269,75.7254,0.0,461.096,1414.070,3584.84,3.04476,30.0692,334.102,735.901,955.86,...,39.16300,529.860,69.0000,846.838,998.153,277.861,681.764,7.17191,319.692,1
HCM273,37.6605,0.0,381.455,978.909,4945.51,4.54240,17.4854,623.545,1153.430,1393.92,...,20.35360,380.116,33.0000,1209.590,906.790,342.850,482.818,25.41640,262.259,1
HCM282,15.6717,0.0,412.791,924.741,7825.49,3.98671,102.7640,776.293,1079.290,1603.84,...,41.65120,378.434,70.2374,1488.450,1502.000,668.593,821.119,9.47747,352.679,1
HCM395,62.8119,0.0,415.544,753.172,4832.22,5.74996,15.9566,665.929,978.295,1489.18,...,16.11320,419.580,35.5533,1097.870,792.813,310.143,505.360,4.90248,252.070,1
HCM405,43.4162,0.0,259.767,1996.450,6873.09,15.72030,40.1066,464.618,1078.700,1514.09,...,24.27800,628.907,56.0000,1264.100,819.821,386.455,570.512,0.00000,312.924,1
HCM420,76.9105,0.0,330.499,1587.000,8744.40,18.51620,25.6886,647.748,1008.360,1535.73,...,15.18900,721.911,68.1148,1445.080,933.564,558.494,754.054,10.36370,328.159,1
HCM429,36.8202,0.0,407.361,2333.360,8326.50,13.49810,43.5763,531.970,1739.180,1708.57,...,4.25859,1062.180,135.3260,1816.310,1120.570,716.375,769.408,33.23560,285.475,1
HCM431,72.8387,0.0,215.389,946.345,5193.19,4.00333,74.5647,498.462,1286.660,1467.32,...,21.12740,571.923,133.1620,1244.810,969.781,555.070,407.106,7.41254,259.748,1
HCM437,35.2979,0.0,225.110,1074.760,4662.64,11.04010,50.9846,738.213,1243.370,1712.30,...,16.47260,446.009,56.7865,1303.170,859.700,367.694,635.984,0.00000,344.483,1
HCM439,62.4998,0.0,305.325,1841.260,8830.77,1.76300,127.7580,682.964,1745.660,2186.58,...,14.75090,1079.600,112.1000,1911.910,995.494,727.070,648.618,14.01370,302.011,1


In [36]:
# merged.drop([ 'sampleID', 'groupID', 'other'], axis=1, inplace=True)
# merged.head()

#导出merge的数据到 /share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/discovery中
merged.to_csv('./validation3/validation3.csv', index=True)
print(merged.tail())
print(merged.shape) 

          VEGFA      FN1       EGF      KDR    ITGB1    NOTCH1     HIF1A  \
ND1_LV  56.2607      0.0   465.524  1522.30  6823.19   7.25553   47.9729   
ND2     72.3104      0.0   562.881  2761.43  7828.77   4.66081   68.9209   
sc2_LV  73.6980      0.0   353.235  2037.26  5584.91   4.31385   98.2407   
sc5_LV  77.0709  14777.6   296.807  2115.40  9969.04   9.39348  110.6470   
sc6_LV  21.8786      0.0  1023.770  3154.22  9776.65  10.65710   31.3361   

             TEK   PDGFRB     FLT1  ...    AIMP1    CALCRL      EGR3     ELK3  \
ND1_LV   792.456  2032.01  2240.93  ...  45.5162   779.755   60.0000  1431.63   
ND2     1463.400  3127.95  2872.77  ...  35.3264  1276.550  105.7430  2258.22   
sc2_LV  1039.930  1748.13  3159.91  ...  63.6294   547.525   73.1772   872.33   
sc5_LV   872.391  1745.51  1245.02  ...  28.9605   353.244   68.7151  1322.56   
sc6_LV  1248.590  2736.20  3656.99  ...  40.8272   795.641  107.7600  1718.03   

          NAA15      RTN4   SLC12A6   THSD7A   ZNF304  g